In [13]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os

pd.options.display.float_format = '{:.0f}'.format # Supress scientific notation

In [14]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip -nv

with ZipFile('book-crossings.zip', 'r') as zObject:
    zObject.extractall()

os.remove('book-crossings.zip')

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
# users_filename = 'BX-Users.csv'

2024-01-11 21:13:56 URL:https://cdn.freecodecamp.org/project-data/books/book-crossings.zip [26085508/26085508] -> "book-crossings.zip" [1]


In [16]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [17]:
df = df_ratings.merge(df_books)

In [18]:
df.head()

,user,isbn,rating,title,author
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose


In [19]:
df.describe()

,user,rating
count,1031175,1031175
mean,140594,3
std,80524,4
min,2,0
25%,70415,0
50%,141210,0
75%,211426,7
max,278854,10


In [20]:
# Get list of users that posted 200+ reviews
users = df.groupby('user').count().isbn.sort_values()
users_few = users.loc[users >= 200].index.to_list()

# Get list of books that have 100+ reviews
books = df.groupby('isbn').count().user.sort_values()
books_few = books.loc[books >= 100].index.to_list()

# Filter dataframe with previously generated lists
df = df.loc[(df.isbn.isin(books_few)) & (df.user.isin(users_few))]

In [21]:
df.describe()

,user,rating
count,48358,48358
mean,141317,2
std,80922,4
min,254,0
25%,70415,0
50%,141493,0
75%,214786,1
max,278418,10


In [10]:
# Convert dataframe into 2D matrix
df = df_ratings.pivot(index='user', columns='isbn', values='rating').fillna(0)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()